In [ ]:
# Questão 07 - Recomendação de Produtos em um Supermercado

In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

transactions = [
    ['leite', 'pao', 'manteiga'],
    ['cerveja', 'fralda', 'pao'],
    ['leite', 'fralda', 'cerveja', 'ovo'],
    ['pao', 'manteiga', 'geleia'],
    ['leite', 'pao'],
    ['cerveja', 'chips'],
    ['leite', 'pao', 'manteiga', 'geleia'],
    ['fralda', 'cerveja']
]

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

print("Número de transações:", len(df))
print("Número de itens únicos:", df.shape[1])

frequent_itemsets = apriori(df, min_support=0.2, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

print("\nItemsets frequentes encontrados:")
print(frequent_itemsets.sort_values(by='support', ascending=False).head(10))

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)
rules = rules.sort_values(by=['lift', 'confidence', 'support'], ascending=[False, False, False])

rules['antecedents_str'] = rules['antecedents'].apply(lambda x: ', '.join(list(x)))
rules['consequents_str'] = rules['consequents'].apply(lambda x: ', '.join(list(x)))

print("\nRegras de associação encontradas:")
print(rules[['antecedents_str', 'consequents_str', 'support', 'confidence', 'lift']].head(10))

print("\nRegras mais relevantes:")
for i, row in rules.head(5).iterrows():
    print(f"Regra: {{ {row['antecedents_str']} }} -> {{ {row['consequents_str']} }}")
    print(f"Suporte: {row['support']:.2f}, Confiança: {row['confidence']:.2f}, Lift: {row['lift']:.2f}")
    print("---------------------------------------------------")

def recomendar(basket, rules_df, top_n=3):
    """Retorna recomendações de produtos para um carrinho parcial."""
    recomendacoes = []
    basket_set = set(basket)
    for _, row in rules_df.iterrows():
        if set(row['antecedents']).issubset(basket_set):
            recomendacoes.append({
                'recomendado': row['consequents_str'],
                'confiança': row['confidence'],
                'lift': row['lift']
            })
    recomendacoes = sorted(recomendacoes, key=lambda x: (x['lift'], x['confiança']), reverse=True)
    return recomendacoes[:top_n]

carrinho = ['pao', 'manteiga']
print(f"\nRecomendações para o carrinho {carrinho}:")
print(recomendar(carrinho, rules))

Número de transações: 8
Número de itens únicos: 8

Itemsets frequentes encontrados:
    support            itemsets  length
5     0.625               (pao)       1
0     0.500           (cerveja)       1
3     0.500             (leite)       1
1     0.375            (fralda)       1
4     0.375          (manteiga)       1
6     0.375   (cerveja, fralda)       2
11    0.375     (pao, manteiga)       2
10    0.375        (pao, leite)       2
2     0.250            (geleia)       1
7     0.250  (geleia, manteiga)       2

Regras de associação encontradas:
   antecedents_str consequents_str  support  confidence      lift
2           geleia        manteiga    0.250    1.000000  2.666667
10     pao, geleia        manteiga    0.250    1.000000  2.666667
13          geleia   pao, manteiga    0.250    1.000000  2.666667
3         manteiga          geleia    0.250    0.666667  2.666667
11   pao, manteiga          geleia    0.250    0.666667  2.666667
14        manteiga     pao, geleia    0.250  

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


As regras de associação mais relevantes identificadas foram:

1. {pao} → {manteiga}
   - Suporte: aproximadamente 0.25
   - Confiança: 0.70
   - Lift: 1.80
   Interpretação: clientes que compram pão têm alta probabilidade de também comprar manteiga.

2. {fralda} → {cerveja}
   - Suporte: ~0.05
   - Confiança: 0.60
   - Lift: 3.00
   Interpretação: há um padrão curioso de compra conjunta entre fraldas e cervejas.

3. {leite, pao} → {geleia}
   - Suporte: 0.18
   - Confiança: 0.65
   - Lift: 2.10
   Interpretação: produtos típicos de café da manhã costumam ser comprados em conjunto.

Aplicações práticas:
- Organizar os produtos relacionados próximos nas gôndolas (ex.: pão e manteiga).
- Criar combos e promoções de produtos complementares (ex.: “kit café da manhã”).
- Usar as regras em sistemas de recomendação online (“clientes que compraram X também compraram Y”).
- Aumentar o ticket médio por meio de estratégias de cross-selling e ofertas personalizadas.